# Analyzing Patterns & Trends In Key Infrastructure Projects In Ontario

## Abstract
 We decided to proceed with the following problem: where is most of the expansion happening across Ontario, which locations have the most investment in infrastructure, how many of these projects finished successfully, and how many are in progress?

 Analyzing where the government is deciding to invest in projects can result with some insights regarding the direction of our province.

 These are just some of the questions we found interesting when presented with this data. The source of the data we used is https://data.ontario.ca/en/dataset/ontario-builds-key-infrastructure-projects.

## Introduction 
A city's overall health can be equated by the amount of projects it completes and by how efficiently and well distributed these projects ultimately are carried out. Ontario being a well known and developed Canadian city, we wish to find out: 
- Where is most of the expansion happening?,
- Which locations have the most investment in infrastructure?, 
- How many projects are completed, undergoing or are in the planning phase and in what categories?
- What categories have the most projects?
- How is funding distributed among these projects?

Infrastructure is the backbone of any city or town. Analyzing where the government is deciding to invest in projects can result with some insights regarding the direction of this province.

These are just some of the questions we found interesting when presented with this data. 


## Gathering data

The source of the data we will be using is found at https://data.ontario.ca/en/dataset/ontario-builds-key-infrastructure-projects.

This data contains all of the current, past and future projects situated within the Ontario region. 

## Prepping and loading the data

In this notebook we will be using python with the following libraries:

In [68]:
import geopandas
import pandas as pd
import pandas_bokeh

The above libraries will help us to manipulate and plot the data in diferent ways for us to easily visualize.

Now we proceed to load the data into this notebook

In [69]:
# This Embeds the Plots in the cell outputs of the notebook.
pandas_bokeh.output_notebook()

# Loading the data into a dataframe and using _id as index
projects = pd.read_csv('projects.csv', ',').set_index('_id')

# Show the data
print(projects)

Loading BokehJS ...

       Category                  Supporting Ministry     Community  \
_id                                                                  
1     Broadband  Agriculture, Food and Rural Affairs   Bracebridge   
2     Broadband  Agriculture, Food and Rural Affairs  Brant County   
3     Broadband  Agriculture, Food and Rural Affairs     Brantford   
4     Broadband  Agriculture, Food and Rural Affairs  Bruce County   
5     Broadband  Agriculture, Food and Rural Affairs       Caledon   
...         ...                                  ...           ...   
1246  Education                            Education   Tillsonburg   
1247  Education                            Education   Tillsonburg   
1248  Education                            Education   Tillsonburg   
1249  Education                            Education   Tillsonburg   
1250  Education                            Education       Timmins   

                                      Project    Status  \
_id                           

The data contains a total of 23 columns which includes a lot of details about all projects within the Ontario region.

## Mapping the data

To find out where is the most expansion happening, we map every single project across Ontario using geopandas and bokeh. Pandas allows us more functionality using geopandas as the backbone. 

In [70]:
# We have to drop all projects with NaN longitude/latitude values, so they will not appear on the map.
projects_filtered = projects[projects['Latitude'].notna()]
projects_filtered = projects[['Project', 'Latitude', 'Longitude', 'Category', 'Estimated Total Budget']]
projects_filtered = projects_filtered.dropna()

# Plot the map with geolocation of projects
projects_filtered.plot_bokeh.map(
    x="Longitude",
    y="Latitude",
    hovertool_string="""<h2> @{Category} </h2>
                       <h3> Project: @{Project}
                       <p> Estimated Total Budget: @{Estimated Total Budget}""",
    tile_provider="CARTODBPOSITRON_RETINA",
    figsize=(1000, 600),
    title="Projects Across Ontario")


Figure(id='9892', ...)

It is immediately noticable that Toronto has a high density of infrastructure plans.

We can view that the bulk of projects are centered around the bigger metropolis like Toronto and Montreal. Concentrating future development to these two cities in particular. 

Mousing over a data point shows the category of the project, as well as the name & estimated total budget.

## Visualizing No. of projects per Ontario region

We can further analize the location of projects by showing the total amount of projects by the region they are in (East, Central, Northwest, Northeast or Southwest) in a pie chart

In [71]:
# Grouping data by region
projects_by_region = pd.DataFrame({'count': projects[['Region']].groupby(['Region']).size()}).reset_index()
projects_by_region = projects_by_region.sort_values(by='Region', ascending=False)
print(projects_by_region)

# Plot the pie chart
p_bar = projects_by_region.plot_bokeh.pie(
    x='Region',
    ylabel="No. of Projects", 
    title="Projects Distribution",
    colormap=["blue", "red", "yellow", "green", "purple", "orange", "grey"],
    stacked=True,
    alpha=0.6)


      Region  count
4  Southwest    237
3  Northwest     66
2  Northeast    157
1       East    286
0    Central    504
__x__values_original


Here we can see that the bulk of projects in Ontario are situated in the 
Central, East and Soutwest regions, wich corroborates the data shown in the previous map

## Visualizing Projects By Category

Here we will visualize the distribution of projects by category. We will be using the number of projects demonstrated with a bar plot. This will help us determine which categories have the most projects assigned to them.

In [72]:
# We count and group all the categories in the data
projects_by_category = pd.DataFrame({'count': projects[['Category']].groupby(['Category']).size()}).reset_index()
# Sort by highest value first
projects_by_category = projects_by_category.sort_values(by='count', ascending=False)
print(projects_by_category)

# Plotting bar plot
p_bar = projects_by_category.plot_bokeh.bar(
    x="Category",
    ylabel="No. of Projects", 
    title="Projects by Category", 
    alpha=0.6)

            Category  count
3          Education    582
1         Child care    342
2        Communities    172
4  Roads and bridges    114
0          Broadband     38
5            Transit      2


From this plot it is clear what the government is prioritizing. Education has the most projects currently, followed by Child Care, Communities, Roads & Bridges, Broadband, and finally Transit.

It seems Ontario prioritizes social projects over infrastructure ones, which I believe will only continue to centralize project count to only certain areas, thus the distribution will continue to be unequal across Ontario.

## Visualizing Projects By Category & Status
To Know how many projects are completed, planned or under construction, we can use a stacked bar plot for quick visualization.
<a id='here'></a>

In [73]:
# Grouping data by Category and Status
projects_by_category_and_status = pd.DataFrame({'count': projects[['Category','Status']].groupby(['Category','Status']).size()}).reset_index()
#print(projects_by_category_and_status)

# Pivot the grouping data to classify by  Status
projects_by_status = projects_by_category_and_status.pivot(index='Category', columns='Status', values='count').reset_index()
# This removes column index name
projects_by_status.columns.name=None
print(projects_by_status)

# Plot the stacked bar plot
p_bar = projects_by_status.plot_bokeh.bar(
    x='Category',
    ylabel="No. of Projects", 
    title="Projects Distribution",
    stacked=True,
    alpha=0.6)


            Category  Complete  Planning  Under construction
0          Broadband       2.0      33.0                 3.0
1         Child care     207.0      93.0                42.0
2        Communities     153.0       4.0                15.0
3          Education     435.0     100.0                47.0
4  Roads and bridges     112.0       NaN                 2.0
5            Transit       2.0       NaN                 NaN


From this plot we can conclude that Broadband and Transit projects have the least priority thus having the least completed projects. Roads and bridges and Transit have no projects planned for the future meaning there is no distribution for them. 
Education, Child care and Communities have the most completed projects, while CC and Education have the most planned and under construcion projects currently. 

It clearly seems Ontario has social projects (i.e. Education, Child Care) as the highest priority projects, while infrastructure (i.e. Transit, Broadband) have the least priority.

## Types of funding per project (federal, municipal, provincial, etc.)
To view how the funding is distributed, we will once again show a bar plot with each type of funding (Municipal, Provincial, Federal, Other).

In [74]:
# Grouping data by funding type
types_of_funding = ['Municipal Funding', 'Provincial Funding', 'Federal Funding', 'Other Funding']
projects_by_funding_type = pd.DataFrame(projects[types_of_funding])
#projects_by_funding_type = projects_by_funding.sort_values(by='Region', ascending=False)

# This replaces Nan values with 0 and Yes values with 1
projects_by_funding_type = projects_by_funding_type.fillna(0)
projects_by_funding_type = projects_by_funding_type.eq('Yes').mul(1)

# Get the totals and create new Dataframe
project_funding_dist = pd.DataFrame({'Municipal Funding':[projects_by_funding_type['Municipal Funding'].sum()],
                    'Provincial Funding':[projects_by_funding_type['Provincial Funding'].sum()],
                    'Federal Funding':[projects_by_funding_type['Federal Funding'].sum()],
                    'Other Funding':[projects_by_funding_type['Other Funding'].sum()]})

# This removes column index name
project_funding_dist.columns.name=None
print(project_funding_dist)

# Plot the bar plot
p_bar = project_funding_dist.plot_bokeh.bar(
    xlabel='projects',
    ylabel="Project total", 
    title="All projects funded",
    #stacked=True,
    alpha=0.6)

   Municipal Funding  Provincial Funding  Federal Funding  Other Funding
0                233                1181              118            335


As we can see here, funding is done mostly bye their provinces, with the least amount of funding coming from Federal sources. 

While there exists some funding overlap between projects, all project get some amount of funding at the provincial level.

## Conclusion

We can now answer all of our initial questions:
- Where is most of the expansion happening?


In the Central, East and Soutwest regions, mainly in the cities of Toronto and Montreal

- Which locations have the most investment in infrastructure?,


Infrastructure projects are very few, but are also in the 
Central, East and Soutwest regions

- How many projects are completed, undergoing or are in the planning phase and in what categories?


It clearly seems Ontario has social projects (i.e. Education, Child Care) as the highest priority projects, while infrastructure (i.e. Transit, Broadband) have the least priority. The amount can be viewed [here](#here)

- What categories have the most projects?


Education has the most projects currently, followed by Child Care, Communities, Roads & Bridges, Broadband, and finally Transit.

It seems Ontario prioritizes social projects over infrastructure ones, which I believe will only continue to centralize project count to only certain areas, thus the distribution will continue to be unequal across Ontario.

- How is funding distributed among these projects?


The majority of funding for every project comes from provincial sources, even wish overlap between sources.